In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import re
import string

import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
nltk.download('stopwords')
nltk.download('vader_lexicon')


from collections import Counter

from matplotlib import pyplot as plt
from matplotlib import ticker
import seaborn as sns
import plotly.express as px

sns.set(style="darkgrid")

[nltk_data] Downloading package stopwords to C:\Users\Arpit
[nltk_data]     Pardesi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Arpit
[nltk_data]     Pardesi\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [14]:
# pip install plotly

In [ ]:
# importing CSV

data = pd.read_csv('https://raw.githubusercontent.com/gabrielpreda/covid-19-tweets/master/covid19_tweets.csv')
data.head()

In [16]:
#dataframe shape
data.shape

(179108, 13)

In [17]:
# Selecting necessary columns

needed_cols=['user_name','text','date',]
df = data[needed_cols]
df.head()

,user_name,text,date
0,ᏉᎥ☻լꂅϮ,If I smelled the scent of hand sanitizers toda...,2020-07-25 12:27:21
1,Tom Basile 🇺🇸,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,2020-07-25 12:27:17
2,Time4fisticuffs,@diane3443 @wdunlap @realDonaldTrump Trump nev...,2020-07-25 12:27:14
3,ethel mertz,@brookbanktv The one gift #COVID19 has give me...,2020-07-25 12:27:10
4,DIPR-J&K,25 July : Media Bulletin on Novel #CoronaVirus...,2020-07-25 12:27:08


In [18]:
#changing column types

df.user_name = df.user_name.astype('category')
df.user_name = df.user_name.cat.codes

df.date = pd.to_datetime(df.date).dt.date
df.head()

F:\Anaconda\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,user_name,text,date
0,89755,If I smelled the scent of hand sanitizers toda...,2020-07-25
1,76403,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,2020-07-25
2,76147,@diane3443 @wdunlap @realDonaldTrump Trump nev...,2020-07-25
3,84572,@brookbanktv The one gift #COVID19 has give me...,2020-07-25
4,18398,25 July : Media Bulletin on Novel #CoronaVirus...,2020-07-25


In [19]:
text = df.text
text

0         If I smelled the scent of hand sanitizers toda...
1         Hey @Yankees @YankeesPR and @MLB - wouldn't it...
2         @diane3443 @wdunlap @realDonaldTrump Trump nev...
3         @brookbanktv The one gift #COVID19 has give me...
4         25 July : Media Bulletin on Novel #CoronaVirus...
                                ...                        
179103    Thanks @IamOhmai for nominating me for the @WH...
179104    2020! The year of insanity! Lol! #COVID19 http...
179105    @CTVNews A powerful painting by Juan Lucena. I...
179106    More than 1,200 students test positive for #CO...
179107    I stop when I see a Stop\n\n@SABCNews\n@Izinda...
Name: text, Length: 179108, dtype: object

In [20]:
# Reoving URLS

remove_url = lambda x: re.sub(r'https\S+','',str(x))
text_lr =text.apply(remove_url)
text_lr

0         If I smelled the scent of hand sanitizers toda...
1         Hey @Yankees @YankeesPR and @MLB - wouldn't it...
2         @diane3443 @wdunlap @realDonaldTrump Trump nev...
3         @brookbanktv The one gift #COVID19 has give me...
4         25 July : Media Bulletin on Novel #CoronaVirus...
                                ...                        
179103    Thanks @IamOhmai for nominating me for the @WH...
179104           2020! The year of insanity! Lol! #COVID19 
179105    @CTVNews A powerful painting by Juan Lucena. I...
179106    More than 1,200 students test positive for #CO...
179107    I stop when I see a Stop\n\n@SABCNews\n@Izinda...
Name: text, Length: 179108, dtype: object

In [21]:
# to lowercase

to_lower = lambda x: x.lower()
text_lr_lc = text_lr.apply(to_lower)
text_lr_lc

0         if i smelled the scent of hand sanitizers toda...
1         hey @yankees @yankeespr and @mlb - wouldn't it...
2         @diane3443 @wdunlap @realdonaldtrump trump nev...
3         @brookbanktv the one gift #covid19 has give me...
4         25 july : media bulletin on novel #coronavirus...
                                ...                        
179103    thanks @iamohmai for nominating me for the @wh...
179104           2020! the year of insanity! lol! #covid19 
179105    @ctvnews a powerful painting by juan lucena. i...
179106    more than 1,200 students test positive for #co...
179107    i stop when i see a stop\n\n@sabcnews\n@izinda...
Name: text, Length: 179108, dtype: object

In [22]:
# removing punctuations

remove_punc = lambda x: x.translate(str.maketrans('','',string.punctuation))
text_lr_lc_np = text_lr_lc.apply(remove_punc)
text_lr_lc_np

0         if i smelled the scent of hand sanitizers toda...
1         hey yankees yankeespr and mlb  wouldnt it have...
2         diane3443 wdunlap realdonaldtrump trump never ...
3         brookbanktv the one gift covid19 has give me i...
4         25 july  media bulletin on novel coronavirusup...
                                ...                        
179103    thanks iamohmai for nominating me for the who ...
179104               2020 the year of insanity lol covid19 
179105    ctvnews a powerful painting by juan lucena its...
179106    more than 1200 students test positive for covi...
179107    i stop when i see a stop\n\nsabcnews\nizindaba...
Name: text, Length: 179108, dtype: object

In [27]:
#removing stopwords

more_words = ['covid','#coronavirus', '#coronavirusoutbreak', '#coronavirusPandemic', '#covid19', '#covid_19', '#epitwitter', '#ihavecorona', 'amp', 'coronavirus', 'covid19']
stop_words = set(stopwords.words('English'))
stop_words.update(more_words)

remove_words = lambda x: ' '.join(word for word in x.split() if word not in stop_words)
text_lr_lc_np_ns = text_lr_lc_np.apply(remove_words)
text_lr_lc_np_ns

0         smelled scent hand sanitizers today someone pa...
1         hey yankees yankeespr mlb wouldnt made sense p...
2         diane3443 wdunlap realdonaldtrump trump never ...
3         brookbanktv one gift give appreciation simple ...
4         25 july media bulletin novel coronavirusupdate...
                                ...                        
179103    thanks iamohmai nominating wearamask challenge...
179104                               2020 year insanity lol
179105    ctvnews powerful painting juan lucena tribute ...
179106    1200 students test positive major university a...
179107    stop see stop sabcnews izindabanews24 dailysun...
Name: text, Length: 179108, dtype: object

In [29]:
# making list of words from all the texts

words_list = [word for line in text_lr_lc_np_ns for word in line.split()]
words_list[:5]

['smelled', 'scent', 'hand', 'sanitizers', 'today']

In [32]:
# most common words

word_counts = Counter(words_list).most_common(50)
word_df = pd.DataFrame(word_counts, columns=['word','freq'])
word_df.head()

,word,freq
0,cases,18596
1,new,14683
2,people,9053
3,pandemic,7869
4,deaths,7429


In [35]:
# Plotting bar graph for Most common Words

px.bar(word_df, x='word', y='freq', title='Most common words')

In [36]:
# Adding cleaned text into main dataframe

data.text = text_lr_lc_np_ns
data.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,smelled scent hand sanitizers today someone pa...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,hey yankees yankeespr mlb wouldnt made sense p...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,diane3443 wdunlap realdonaldtrump trump never ...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,brookbanktv one gift give appreciation simple ...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 july media bulletin novel coronavirusupdate...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False
